This notebook demonstrates the inference for text summarization using Falcon-7b-instruct model.

Getting inference mainly based on defining three steps: model, tokenizer, and pipeline.

In [3]:
!pip install transformers

In [4]:
import transformers
from dola import DoLa

model_name = "huggyllama/llama-7b"
device = "cuda"
num_gpus = 1
max_gpu_memory = 27
dola_model = DoLa(model_name, device, num_gpus, max_gpu_memory)

model = transformers.AutoModelForCausalLM.from_pretrained(
  'huggyllama/llama-7b',
  trust_remote_code=True
)
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("huggyllama/llama-7b")



Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.68s/it]


In [5]:
# from transformers import pipeline
# pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, max_new_tokens=100, do_sample=True, use_cache=True, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.eos_token_id)
# from langchain import HuggingFacePipeline
# llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0.1})

In [6]:
# from langchain import PromptTemplate,  LLMChain

# template = """
#               Write a concise summary of the following text delimited by triple backquotes.
#               ```{text}```
#               SUMMARY:
#            """

# prompt = PromptTemplate(template=template, input_variables=["text"])

# llm_chain = LLMChain(prompt=prompt, llm=llm)

In [7]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset('cnn_dailymail', '3.0.0')
import pandas as pd
test_df = pd.DataFrame(dataset['test'])

In [6]:
test_df.head()
# test_df.drop(columns=['id'], inplace=True)

article  \
0  (CNN)The Palestinian Authority officially beca...   
1  (CNN)Never mind cats having nine lives. A stra...   
2  (CNN)If you've been following the news lately,...   
3  (CNN)Five Americans who were monitored for thr...   
4  (CNN)A Duke student has admitted to hanging a ...   

                                          highlights  \
0  Membership gives the ICC jurisdiction over all...   
1  Theia, a bully breed mix, was apparently hit b...   
2  Mohammad Javad Zarif has spent more time with ...   
3  17 Americans were exposed to the Ebola virus w...   
4  Student is no longer on Duke University campus...   

                                         id  
0  f001ec5c4704938247d27a44948eebb37ae98d01  
1  230c522854991d053fe98a718b1defa077a8efef  
2  4495ba8f3a340d97a9df1476f8a35502bcce1f69  
3  a38e72fed88684ec8d60dd5856282e999dc8c0ca  
4  c27cf1b136cc270023de959e7ab24638021bc43f

In [7]:
test_df.drop(columns=['id'], inplace=True)

In [8]:
test_df.head()

article  \
0  (CNN)The Palestinian Authority officially beca...   
1  (CNN)Never mind cats having nine lives. A stra...   
2  (CNN)If you've been following the news lately,...   
3  (CNN)Five Americans who were monitored for thr...   
4  (CNN)A Duke student has admitted to hanging a ...   

                                          highlights  
0  Membership gives the ICC jurisdiction over all...  
1  Theia, a bully breed mix, was apparently hit b...  
2  Mohammad Javad Zarif has spent more time with ...  
3  17 Americans were exposed to the Ebola virus w...  
4  Student is no longer on Duke University campus...

In [9]:
test_df['article'][0]

'(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC\'s founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians\' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday\'s ceremony

In [10]:
test_df['highlights'][0]

'Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June .\nIsrael and the United States opposed the move, which could open the door to war crimes investigations against Israelis .'

In [16]:
def load_csv():
    # Format of each line:
    # {'instruction': ..., 'input': ..., 'output':...}
    '''
    Data format:

    ,full_prefix,doc_id,completion,contradiction_0,contradiction_1,contradiction_2,longest_completions,turncated_prefixes
    0,"As streaming television services continue to gain market share, there are a number of reasons why Netflix might be in trouble. Time Warner is taking its HBO content online, Amazon offers premium content for a monthly fee, and Hulu has reached nine million users. While these competitors may cause a bit of worry, it’s not the end of the world. Although Netflix has a huge amount of potential, the increased competition is unlikely to hurt its profitability.
    While the global pandemic last year caused a major shakeup in Hollywood, Netflix should not rest on its laurels. With a variety of rivals on the rise, it’s unlikely that it can continue to rely on its current performance. Despite the competition, the company has made a number of impactful moves across the board, including clamping down on password sharing. And in the coming years, Netflix should continue to grow and compete with new competitors.
    With more competitors entering the streaming space, Netflix is likely to face a more difficult time keeping its current market share. Disney has been investing heavily in the service and Amazon is expected to do the same. Both companies expect to add 35-40 million subscribers per year through 2024. Despite the competition, Netflix still remains the top streaming service. Its lack of original content has hurt its numbers in the last few quarters. Its only big original hit in the US was Cobra Kai, which only got four seasons. ",0,Whether or not it gets a second season of The Witcher is another question.,Whether or not it gets a second season of Stranger Things is another question.,Whether or not it gets a fifth season of The Witcher is another question.,Whether or not it gets a second season of Black Mirror is another question.,15.0,"As streaming television services continue to gain market share, there are a number of reasons why Netflix might be in trouble. Time Warner is taking its HBO content online, Amazon offers premium content for a monthly fee, and Hulu has reached nine million users. While these competitors may cause a bit of worry, it’s not the end of the world. Although Netflix has a huge amount of potential, the increased competition is unlikely to hurt its profitability.
    While the global pandemic last year caused a major shakeup in Hollywood, Netflix should not rest on its laurels. With a variety of rivals on the rise, it’s unlikely that it can continue to rely on its current performance. Despite the competition, the company has made a number of impactful moves across the board, including clamping down on password sharing. And in the coming years, Netflix should continue to grow and compete with new competitors.
    With more competitors entering the streaming space, Netflix is likely to face a more difficult time keeping its current market share. Disney has been investing heavily in the service and Amazon is expected to do the same. Both companies expect to add 35-40 million subscribers per year through 2024. Despite the competition, Netflix still remains the top streaming service. Its lack of original content has hurt its numbers in the last few quarters. Its only big original hit in the US was Cobra Kai, which only got four seasons. "

    '''
    list_data_dict = []
    df = test_df

    prefix_type = 'article'
    for idx in range(len(df)):
        item = dict(
            prefix=df[prefix_type][idx],
            completion=df['highlights'][idx],
        )
        list_data_dict.append(item)
    return list_data_dict

In [27]:
import os


list_data_dict = load_csv()

llm = DoLa(model_name, device, num_gpus, max_gpu_memory)
# llm.set_stop_words(["\n", "(CNN)", "\end{code}"])

Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.31s/it]


In [32]:
#generating the summary:
from tqdm import tqdm

test_df['model_generated'] = ""

# Define a function to generate summaries and populate the 'model_generated' column
def generate_and_store_summary(row):
    article_text = row['article']
    article_text = """
              Write a concise summary of the following text delimited by triple backquotes in 5 lines.
              ```""" + article_text + """```
              SUMMARY:
           """
    summary = llm.generate(article_text, mature_layer = 32, candidate_premature_layers = [0,2,4,6,8,10,12,14], mode='dola')
    return summary
# Apply the generate_and_store_summary function to generate summaries for the first 25 articles
test_df.loc[:24, 'model_generated'] = test_df.loc[:24].apply(generate_and_store_summary, axis=1)

# # Display the updated DataFrame with generated summaries for the first 25 articles
print(test_df[['article', 'model_generated']].head(25))

MODEL OUTPUT: 
```
              Palestine officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday's 

In [33]:
test_df.head()

article  \
0  (CNN)The Palestinian Authority officially beca...   
1  (CNN)Never mind cats having nine lives. A stra...   
2  (CNN)If you've been following the news lately,...   
3  (CNN)Five Americans who were monitored for thr...   
4  (CNN)A Duke student has admitted to hanging a ...   

                                          highlights  \
0  Membership gives the ICC jurisdiction over all...   
1  Theia, a bully breed mix, was apparently hit b...   
2  Mohammad Javad Zarif has spent more time with ...   
3  17 Americans were exposed to the Ebola virus w...   
4  Student is no longer on Duke University campus...   

                                         id  \
0  f001ec5c4704938247d27a44948eebb37ae98d01   
1  230c522854991d053fe98a718b1defa077a8efef   
2  4495ba8f3a340d97a9df1476f8a35502bcce1f69   
3  a38e72fed88684ec8d60dd5856282e999dc8c0ca   
4  c27cf1b136cc270023de959e7ab24638021bc43f   

                                     model_generated  
0  (```\n              Palestine officially becam...  
1  (A stray pooch in Washington State has used up...  
2  (```\n              Mohammad Javad Zarif is th...  
3  (```\n              Five Americans who were mo...  
4  (A Duke student has admitted to hanging a noos...

In [41]:
test_df.iloc[2,:]['article']

'(CNN)If you\'ve been following the news lately, there are certain things you doubtless know about Mohammad Javad Zarif. He is, of course, the Iranian foreign minister. He has been U.S. Secretary of State John Kerry\'s opposite number in securing a breakthrough in nuclear discussions that could lead to an end to sanctions against Iran -- if the details can be worked out in the coming weeks. And he received a hero\'s welcome as he arrived in Iran on a sunny Friday morning. "Long live Zarif," crowds chanted as his car rolled slowly down the packed street. You may well have read that he is "polished" and, unusually for one burdened with such weighty issues, "jovial." An Internet search for "Mohammad Javad Zarif" and "jovial" yields thousands of results. He certainly has gone a long way to bring Iran in from the cold and allow it to rejoin the international community. But there are some facts about Zarif that are less well-known. Here are six: . In September 2013, Zarif tweeted "Happy Rosh

In [42]:
test_df.iloc[2,:]['model_generated']

('```\n              Mohammad Javad Zarif is the Iranian foreign minister. He has been U.S. Secretary of State John Kerry\'s opposite number in securing a breakthrough in nuclear discussions that could lead to an end to sanctions against Iran -- if the details can be worked out in the coming weeks. And he received a hero\'s welcome as he arrived in Iran on a sunny Friday morning. "Long live Zarif," crowds chanted as his car rolled slowly down the packed street. You may well have read that he is "polished" and, unusually for one burdened with such weighty issues, "jovial." An Internet search for "Mohammad Javad Zarif" and "jovial" yields thousands of results. He certainly has gone a long way to bring Iran in from the cold and allow it to rejoin the international community. But there are some facts about Zarif that are less well-known. Here are six: . In September 2013, Zarif tweeted "Happy Rosh Hashanah," referring to the Jewish New Year. That prompted Christine Pelosi, the daughter of 

Randomly visualization of generated text

In [14]:
test_df['model_generated'][11]

' ```\n              Text in bold.\n            ```\n              Detailed Summary:\n             ```text in italics ```\n              ```A few sentences about what happened to Paul Walker. He got into a car accident, and died at age 40. He is well loved in the movie industry, for his car movies and most recently his action movies. His fans have made tribute to him online and offline. For example people named after him, or people saying that he'

In [15]:
test_df['model_generated'][24]

' Summary:\n        """\n        self.cnn_tweets_url = self.COND_URL_GET % params\n        self.CNN_POST = self.COND_URL_POST\n\n        # CNN POST\n        self.cnn_tweet_api_code = self.get_cnn_tweet_by_api_code_from_url(self.CNN_POST,\n                                                            self.'

In [36]:
!pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [49]:
test_df['model_generated'].values[0][0]

test_df['model_generated'] = test_df['model_generated'].apply(lambda x: x if type(x)== str else x[0])

In [50]:
from rouge import Rouge

# Initialize the ROUGE evaluator
rouge = Rouge()

# Select the first 25 rows of your DataFrame for evaluation
num_samples = 25
sampled_df = test_df.head(num_samples)

# Extract the generated summaries and reference summaries for the selected samples
generated_summaries = sampled_df['model_generated'].tolist()
reference_summaries = sampled_df['highlights'].tolist()

print(generated_summaries)
print(reference_summaries)

# Calculate ROUGE scores for the selected samples
rouge_scores = rouge.get_scores(generated_summaries, reference_summaries, avg=True)

# Print the ROUGE scores
print("ROUGE Scores:", rouge_scores)


['```\n              Palestine officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC\'s founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians\' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday\'s ceremony,

In [51]:
from nltk.translate.bleu_score import corpus_bleu

# Select the first 25 rows of your DataFrame for evaluation
num_samples = 25
sampled_df = test_df.head(num_samples)

# Extract the generated summaries and reference summaries for the selected samples
generated_summaries = sampled_df['model_generated'].tolist()
reference_summaries = sampled_df['highlights'].tolist()

# Calculate BLEU score for the selected samples
bleu_score = corpus_bleu(reference_summaries, generated_summaries)
print("BLEU Score for 25 Summaries:", bleu_score)


BLEU Score for 25 Summaries: 7.649308833340803e-232


/home/lbham003/anaconda3/envs/dola/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/lbham003/anaconda3/envs/dola/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/lbham003/anaconda3/envs/dola/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider 

In [52]:
!pip install bert_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [53]:
from bert_score import score
# Select the first 25 rows of your DataFrame for evaluation
num_samples = 25
sampled_df = test_df.head(num_samples)

# Extract the generated summaries and reference summaries for the selected samples
generated_summaries = sampled_df['model_generated'].tolist()
reference_summaries = sampled_df['highlights'].tolist()

# Calculate BERT Score
P, R, F1 = score(generated_summaries, reference_summaries, lang="en", verbose=True)

# Print BERT Score
print("BERT Precision:", P.mean().item())
print("BERT Recall:", R.mean().item())
print("BERT F1 Score:", F1.mean().item())


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:00<00:00,  1.48it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 31.04it/s]

done in 0.73 seconds, 34.09 sentences/sec
BERT Precision: 0.7987020611763
BERT Recall: 0.8760218620300293
BERT F1 Score: 0.8351616859436035


In [12]:
# Run this only if you want to run the model for all the samples in the dataset.
# This might take a while on a single GPU and limited memory.
# Run this for fun with multiple GPUs
# Uncomment the code below and run it then

# for sample in tqdm(list_data_dict):
#     context = sample['prefix']
#     context = """
#               Write a concise summary of the following text delimited by triple backquotes in 5 lines.
#               ```""" + context + """```
#               SUMMARY:
#            """
#     # summary = llm.generate(context, mature_layer = 32, candidate_premature_layers = [0,2,4,6,8,10,12,14], mode='dola')
#     summary = llm.generate(context)
#     print('Article: ', context)
#     print('Summary: ', summary)
# # Create an empty column 'model_generated' in test_df to store the generated summaries

                                              article  \
0   (CNN)The Palestinian Authority officially beca...   
1   (CNN)Never mind cats having nine lives. A stra...   
2   (CNN)If you've been following the news lately,...   
3   (CNN)Five Americans who were monitored for thr...   
4   (CNN)A Duke student has admitted to hanging a ...   
5   (CNN)He's a blue chip college basketball recru...   
6   (CNN)Governments around the world are using th...   
7   (CNN)Andrew Getty, one of the heirs to billion...   
8   (CNN)Filipinos are being warned to be on guard...   
9   (CNN)For the first time in eight years, a TV l...   
10  London (CNN)A 19-year-old man was charged Wedn...   
11  (CNN)Paul Walker is hardly the first actor to ...   
12  (CNN)Seventy years ago, Anne Frank died of typ...   
13  (CNN)A year ago Bloomberg published a story wi...   
14  (CNN)If you're famous and performing the Ameri...   
15  (CNN)As goes Walmart, so goes the nation? Ever...   
16  (CNN)On May 28, 2014, some 